In [106]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser

In [107]:
executable_path = {'executable_path':'chromedriver.exe'}
browser=Browser('chrome',**executable_path,headless=False)

In [108]:
url='https://mars.nasa.gov/news'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')

In [109]:
classlists=soup.find_all(class_="list_text")
titletext=[]
paratext=[]
for classlist in classlists:
    titletext.append((classlist.find_all(class_='content_title'))[0].a.text)
    paratext.append((classlist.find_all(class_='article_teaser_body'))[0].text)

In [110]:
url1='https://www.jpl.nasa.gov'
url2='/spaceimages/?search=&category=Mars'
browser.visit((url1+url2))
html1=browser.html
button = browser.find_by_id('full_image')
button.click()
soup1=bs(html1,'html.parser')
imgsrc=soup1.footer.a['data-fancybox-href']
featured_image_url=url1+imgsrc
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17449_ip.jpg'

In [111]:
url='https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html=browser.html
soup=bs(html,'html.parser')
mars_weather=soup.find('p',class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
mars_weather

'InSight sol 394 (2020-01-05) low -99.0ºC (-146.1ºF) high -17.9ºC (-0.3ºF)\nwinds from the SE at 6.6 m/s (14.7 mph) gusting to 23.3 m/s (52.0 mph)\npressure at 6.40 hPapic.twitter.com/R0dFa028si'

In [112]:
url2='https://space-facts.com/mars/'
browser.visit(url2)
html=browser.html
soup=bs(html,'html.parser')
table=soup.find_all('table')[0]

In [113]:
tabledata=pd.read_html(str(table))
tabledf=tabledata[0]

In [114]:
tablehtml=tabledf.to_html()

In [115]:
tabledf

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [116]:
url3='https://astrogeology.usgs.gov'
url4='/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url3+url4)
html2=browser.html
soup=bs(html2,'html.parser')

In [117]:
marshemi=soup.find_all('a',class_='itemLink product-item')

In [118]:
len(marshemi)

8

In [119]:
urllist=[]
for i in [1,3,5,7]:
    urllist.append(url3+marshemi[i]['href'])
urllist

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [120]:
browser.visit(urllist[j])
html1=browser.html
soup1=bs(html1,'html.parser')
imgsrc=soup1.find_all('img',class_='wide-image')[0]['src']
imgsrc

'/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'

In [121]:
hemisphere_image_urls = [
    {"title": "", "img_url": ""},
    {"title": "", "img_url": ""},
    {"title": "", "img_url": ""},
    {"title": "", "img_url": ""},
]

for j in range(len(urllist)):
    
    browser.visit(urllist[j])
    html1=browser.html
    soup1=bs(html1,'html.parser')
    imgsrc=soup1.find_all('img',class_='wide-image')[0]['src']
    hemisphere_image_urls[j]['img_url']=url3+imgsrc
    hemisphere_image_urls[j]['title']=marshemi[((j*2)+1)].h3.text
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [122]:
browser.quit()

In [123]:
results_dict = {"Mars_News_Titles":titletext,
        "Mars_News_Paragraphs":paratext,
        "Main_Image":featured_image_url,
        "Mars_Weather":mars_weather,
        "Mars_Details_inTablehtml":tablehtml,
        "Hemisphere_Images":hemisphere_image_urls
        }

In [134]:
from flask import Flask
from flask_pymongo import PyMongo 
app=Flask(__name__)
mongo = PyMongo(app, uri="mongodb://localhost:27017/mars_app")
import pymongo

In [136]:
mongo.db.collections.update({}, results_dict, upsert=True)

C:\Users\ARJUNJohri\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1,
 'nModified': 0,
 'upserted': ObjectId('5e139aa827d1999ce23b0c64'),
 'ok': 1.0,
 'updatedExisting': False}

In [135]:
conn='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db= client.mars_db
collection = db.mars



In [137]:
result=db.mars.find()

In [139]:
result